In [ ]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager
from src.utils.context import Context

from utils.utils import *

In [2]:
# import torch
# torch.manual_seed(0)

# import random
# random.seed(0)

# import numpy as np
# np.random.seed(0)

In [4]:
# Set patient and record (selection based on larger number of observations, more seizure events, and consistent channel configurations)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
# observations = ["chb10_12", "chb10_20", "chb10_27", "chb10_30", "chb10_38", "chb10_89"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN06_2", "PN06_3", "PN06_4", "PN06_5"]
# observations = ["PN14_1", "PN14_2", "PN14_4"]

create_dataset_json(observations)

In [ ]:
logs_path = "./output/logs/"
eval_manager_path = "./output/eval_manager/"

#### Removing cache data

In [ ]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            # Checks if the file begins with "EEG"
            if file.startswith("EEG"):
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

#### Run Gretel experiment

In [ ]:
path = 'config/EEG-CORR-topk.jsonc'
# path = 'config/EEG-CORR-topk-ablation.jsonc'

context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Evaluating records: {observations}")
context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
eval_manager = PairedEvaluatorManager(context)

eval_manager.evaluate()

In [11]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)